# Get_Original_AlphaNet_Data

In [4]:
Alpha_Name = "AlphaNet_Original_Input_12.22_1d_return"
path = '/home/wuwenjun/Data/' + Alpha_Name +'/'
target = "1d_ret"

In [11]:
import time
import os
from Research.backtest.bt import BTDaily
from Research.feature.ft import FeatureAnalysis
import Research.utils.namespace as namespace
import Research.utils.normalization as norm
from Platform.database.mysql import MysqlAPI
from Platform.utils.persistence import convert_to_standard_daily_feature_csv, convert_to_standard_daily_feature_par
from Platform.config.mysql_info import FACTOR_LIB_MYSQL_TIO
import DataAPI
from os import walk
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from progressbar import ProgressBar
from tqdm import tqdm
import multiprocessing as mp

# Get_Original_AlphaNet_Data

In [8]:
dir_list = ['Data','Cov','Corr','Return','Std','Zscore','Decay']
for i in dir_list:
    os.makedirs(path+i)

In [12]:
df = pd.read_csv(path+"data.csv")
df.dropna(inplace=True,axis=0,how="any")
df

,timestamp,ticker,open,high,low,close,vwap,volume,pct_chg,turnover,free_turnover,1d_ret
0,2015-01-06 09:30:00,1,1136.15,1156.75,1108.44,1138.28,1134.052643,2860436.43,1.1364,2.9079,5.7657,-0.018297
1,2015-01-06 09:30:00,2,1774.23,1885.20,1753.27,1838.34,1823.031224,6560835.70,7.2662,6.7834,8.1741,-0.023288
2,2015-01-06 09:30:00,4,63.31,64.62,62.18,63.76,63.277989,28095.12,0.7707,3.3496,4.5343,0.051118
3,2015-01-06 09:30:00,5,38.00,38.00,38.00,38.00,37.997172,0.00,0.0000,0.0000,0.0000,0.000000
4,2015-01-06 09:30:00,6,225.64,234.85,223.10,225.00,228.438278,1078647.12,0.4255,8.0562,13.2751,-0.028571
...,...,...,...,...,...,...,...,...,...,...,...,...
5357756,2021-07-01 09:30:00,688700,47.80,54.50,47.03,53.38,50.715300,64336.83,14.5494,19.2222,19.2222,-0.040395
5357757,2021-07-01 09:30:00,688777,90.21,95.08,89.32,94.42,93.250464,19001.61,4.6111,4.6288,4.6288,-0.005927
5357758,2021-07-01 09:30:00,688788,138.07,149.35,137.11,146.22,145.197985,13572.56,5.1954,7.7977,7.7977,-0.025238
5357759,2021-07-01 09:30:00,688819,44.04,45.64,43.95,45.15,45.026221,47249.01,1.8544,4.6754,4.6754,-0.022042


In [13]:
# for ticker in tqdm(df['ticker'].drop_duplicates()):
def test(ticker):
    one_data = df[df['ticker'] == ticker]
    one_data.set_index(['timestamp','ticker'],inplace=True)
    target_df = one_data[target]
    one_data.drop(target,axis=1,inplace=True)
    concat_df = pd.DataFrame()
    for day in range(29,-1,-1):
        concat_df = pd.concat([concat_df,one_data.shift(day)],axis=1)
    concat_df.dropna(axis=0,how="any",inplace=True)
    concat_df = pd.concat([concat_df,target_df],axis=1,join='inner')
    return concat_df

In [14]:

if __name__=="__main__":
    pool = mp.Pool(10)
    result = {}
    t1 = time.time()
    for ticker in tqdm(df['ticker'].drop_duplicates()):
        result[ticker] = (pool.apply_async(test, args=(ticker,)))   #维持执行的进程总数为10，当一个进程执行完后启动一个新进程.
    pool.close()
    pool.join()
    t2 = time.time()
    print("并行执行时间：", int(t2 - t1))
    df_list = []
    for i in tqdm(result):
        df_list.append(result[i].get())
    final_df = pd.concat(df_list)
final_df

100%|██████████| 4427/4427 [00:00<00:00, 358265.49it/s]

并行执行时间： 50


,,open,high,low,close,vwap,volume,pct_chg,turnover,free_turnover,open,...,open,high,low,close,vwap,volume,pct_chg,turnover,free_turnover,1d_ret
timestamp,ticker,,,,,,,,,,,,,,,,,,,,,
2015-02-16 09:30:00,1,1136.15,1156.75,1108.44,1138.28,1134.052643,2860436.43,1.1364,2.9079,5.7657,1126.20,...,989.78,1007.54,983.38,991.20,996.094771,887743.10,0.6494,0.9025,1.7894,0.000717
2015-02-17 09:30:00,1,1126.20,1164.57,1104.89,1121.23,1132.652885,2166421.40,-1.4981,2.2024,4.3668,1105.60,...,991.20,995.46,975.57,989.07,985.109868,721202.39,-0.2151,0.7332,1.4537,0.005731
2015-02-25 09:30:00,1,1105.60,1124.78,1087.12,1099.91,1101.168988,1700120.67,-1.9011,1.7283,3.4269,1101.33,...,991.91,1002.57,989.78,994.04,996.570831,680491.59,0.5029,0.6918,1.3717,-0.017094
2015-02-26 09:30:00,1,1101.33,1106.31,1058.70,1062.96,1074.097527,1407714.21,-3.3592,1.4311,2.8375,1058.70,...,997.59,998.30,977.70,980.54,983.482739,577392.33,-1.3581,0.5870,1.1638,0.018841
2015-02-27 09:30:00,1,1058.70,1127.62,1045.20,1071.49,1086.375607,2508500.23,0.8021,2.5501,5.0564,1056.57,...,980.54,1000.44,966.33,999.73,986.644628,1025544.24,1.9565,1.0426,2.0672,-0.002134
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-06-30 09:30:00,605339,20.38,24.45,20.38,24.45,24.423600,4472.57,43.9929,0.7040,0.7040,26.90,...,44.75,45.44,44.10,44.99,44.862500,43863.72,0.6938,6.9045,6.9045,-0.034144
2021-07-01 09:30:00,605339,26.90,26.90,26.90,26.90,26.900000,1373.52,10.0204,0.2162,0.2162,29.59,...,44.81,44.88,42.70,43.18,43.469700,72397.12,-4.0231,11.3958,11.3958,0.000231
2021-06-30 09:30:00,688314,100.20,105.00,86.78,92.05,92.899300,106963.24,430.8535,90.5821,90.5821,94.00,...,115.11,115.78,110.21,112.00,112.765600,6286.46,-4.1014,5.3237,5.3237,-0.055645


In [15]:
final_df.columns = [str(i) for i in range(final_df.shape[1])]
final_df.rename(columns={str(final_df.shape[1] - 1):'target'}, inplace=True)
final_df.reset_index(inplace=True)
final_df['timestamp'] = pd.to_datetime(final_df['timestamp'])
final_df.dropna(axis=0,inplace=True) # delete none
final_df

,timestamp,ticker,0,1,2,3,4,5,6,7,...,261,262,263,264,265,266,267,268,269,target
0,2015-02-16 09:30:00,1,1136.15,1156.75,1108.44,1138.28,1134.052643,2860436.43,1.1364,2.9079,...,989.78,1007.54,983.38,991.20,996.094771,887743.10,0.6494,0.9025,1.7894,0.000717
1,2015-02-17 09:30:00,1,1126.20,1164.57,1104.89,1121.23,1132.652885,2166421.40,-1.4981,2.2024,...,991.20,995.46,975.57,989.07,985.109868,721202.39,-0.2151,0.7332,1.4537,0.005731
2,2015-02-25 09:30:00,1,1105.60,1124.78,1087.12,1099.91,1101.168988,1700120.67,-1.9011,1.7283,...,991.91,1002.57,989.78,994.04,996.570831,680491.59,0.5029,0.6918,1.3717,-0.017094
3,2015-02-26 09:30:00,1,1101.33,1106.31,1058.70,1062.96,1074.097527,1407714.21,-3.3592,1.4311,...,997.59,998.30,977.70,980.54,983.482739,577392.33,-1.3581,0.5870,1.1638,0.018841
4,2015-02-27 09:30:00,1,1058.70,1127.62,1045.20,1071.49,1086.375607,2508500.23,0.8021,2.5501,...,980.54,1000.44,966.33,999.73,986.644628,1025544.24,1.9565,1.0426,2.0672,-0.002134
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5230281,2021-06-30 09:30:00,605339,20.38,24.45,20.38,24.45,24.423600,4472.57,43.9929,0.7040,...,44.75,45.44,44.10,44.99,44.862500,43863.72,0.6938,6.9045,6.9045,-0.034144
5230282,2021-07-01 09:30:00,605339,26.90,26.90,26.90,26.90,26.900000,1373.52,10.0204,0.2162,...,44.81,44.88,42.70,43.18,43.469700,72397.12,-4.0231,11.3958,11.3958,0.000231
5230283,2021-06-30 09:30:00,688314,100.20,105.00,86.78,92.05,92.899300,106963.24,430.8535,90.5821,...,115.11,115.78,110.21,112.00,112.765600,6286.46,-4.1014,5.3237,5.3237,-0.055645
5230284,2021-07-01 09:30:00,688314,94.00,106.12,83.00,102.50,92.323600,80295.81,11.3525,67.9987,...,112.32,112.32,103.50,105.64,107.011300,9925.98,-5.6786,8.4058,8.4058,0.008768


In [20]:
import datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta
time_list = [20150101,20150301,20150601,20150901,
             20160101,20160301,20160601,20160901,
             20170101,20170301,20170601,20170901,
             20180101,20180301,20180601,20180901,
             20190101,20190301,20190601,20190901,
             20200101,20200301,20200601,20200901,
             20210101,20210301,20210601]

def save(start_time,end_time):
    final_df[
        (final_df['timestamp'] >= start_time) & 
        (final_df['timestamp'] < end_time)].to_parquet(
        path + "Data/" + datetime.datetime.strptime(str(start_time), '%Y-%m-%d %H:%M:%S').strftime('%Y-%m-%d') + "_" +
                      datetime.datetime.strptime(str(end_time), '%Y-%m-%d %H:%M:%S').strftime('%Y-%m-%d') + ".par"
    )
    print("Save to ",path + "Data/" + datetime.datetime.strptime(str(start_time), '%Y-%m-%d %H:%M:%S').strftime('%Y-%m-%d') + "_" +
                      datetime.datetime.strptime(str(end_time), '%Y-%m-%d %H:%M:%S').strftime('%Y-%m-%d') + ".par")
    return None

pool = mp.Pool(10)
t1 = time.time()
for i in tqdm(range(len(time_list)-1)):
    start_time = pd.to_datetime(str(time_list[i]))
    end_time = pd.to_datetime(str(time_list[i+1]))
    pool.apply_async(save, args=(start_time,end_time))  #维持执行的进程总数为10，当一个进程执行完后启动一个新进程.
pool.close()
pool.join()
t2 = time.time()
print("并行执行时间：", int(t2 - t1))

100%|██████████| 26/26 [00:00<00:00, 3323.94it/s]


Save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Data/2015-01-01_2015-03-01.par
Save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Data/2016-01-01_2016-03-01.par
Save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Data/2017-01-01_2017-03-01.par
Save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Data/2015-03-01_2015-06-01.par
Save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Data/2015-06-01_2015-09-01.par
Save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Data/2017-03-01_2017-06-01.par
Save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Data/2015-09-01_2016-01-01.par
Save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Data/2016-06-01_2016-09-01.par
Save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Data/2016-03-01_2016-06-01.par
Save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Data/2016-09-01_2017-01-01.par


# Get Factor

In [21]:
day = 30
stride = 10
def generateC(l1):
    if len(l1) == 1:
        return []
    v = [[l1[0], i] for i in l1[1:]]
    l1 = l1[1:]
    return v + generateC(l1)

def generate_Num_and_ReversedNum(feat_nums):
    list1 = list(range(feat_nums))
    num = generateC(list1)
    num_rev = []
    for l in num:
        l1 = l.copy()
        l1.reverse()
        num_rev.append(l1)
    return num, num_rev

def generate_Step_List(data_length, stride):
    if data_length % stride == 0:
        step_list = list(range(0, data_length + stride, stride))
    elif data_length % stride <= 5:
        mod = data_length % stride
        step_list = list(range(0, data_length - stride, stride)) + [data_length]
    else:
        mod = data_length % stride
        step_list = list(range(0, data_length + stride - mod, stride)) + [data_length]
    return step_list

In [22]:
path_list = []
data_path = path + "Data/"
dataframe_list = pd.DataFrame()
for f, _, i in walk(data_path):
    for j in tqdm(i):
        path_list.append(j)
path_list.sort()
path_list

100%|██████████| 26/26 [00:00<00:00, 147766.81it/s]


['2015-01-01_2015-03-01.par',
 '2016-01-01_2016-03-01.par',
 '2017-01-01_2017-03-01.par',
 '2015-06-01_2015-09-01.par',
 '2015-03-01_2015-06-01.par',
 '2016-03-01_2016-06-01.par',
 '2016-06-01_2016-09-01.par',
 '2015-09-01_2016-01-01.par',
 '2017-03-01_2017-06-01.par',
 '2016-09-01_2017-01-01.par',
 '2017-06-01_2017-09-01.par',
 '2018-01-01_2018-03-01.par',
 '2017-09-01_2018-01-01.par',
 '2018-03-01_2018-06-01.par',
 '2018-06-01_2018-09-01.par',
 '2018-09-01_2019-01-01.par',
 '2019-01-01_2019-03-01.par',
 '2019-03-01_2019-06-01.par',
 '2019-06-01_2019-09-01.par',
 '2020-01-01_2020-03-01.par',
 '2020-03-01_2020-06-01.par',
 '2019-09-01_2020-01-01.par',
 '2020-06-01_2020-09-01.par',
 '2021-01-01_2021-03-01.par',
 '2020-09-01_2021-01-01.par',
 '2021-03-01_2021-06-01.par']

In [23]:
# Read Data

def cov(x,num,num_rev,step_list):
    """Main Function"""
    l = []
    for i in tqdm(range(len(step_list) - 1)):
        start = step_list[i]
        end = step_list[i + 1]
        sub_data1 = x[:, :, num, start:end]
        sub_data2 = x[:, :, num_rev, start:end]
        mean1 = sub_data1.mean(axis=4, keepdims=True)  # (2000, 1, 36, 2, 1)
        mean2 = sub_data2.mean(axis=4, keepdims=True)
        spread1 = sub_data1 - mean1  # (2000, 1, 36, 2, 10)
        spread2 = sub_data2 - mean2
        cov = ((spread1 * spread2).sum(axis=4, keepdims=True) / (sub_data1.shape[4] - 1)).mean(axis=3,
                                                                                                keepdims=True)  # (2000, 1, 36, 1, 1)
        l.append(cov)
    return np.squeeze(np.array(l)).transpose(1, 2, 0)

def corr(x,num,num_rev,step_list):
    """Main Function"""
    l = []
    for i in tqdm(range(len(step_list) - 1)):
        start = step_list[i]
        end = step_list[i + 1]
        sub_data1 = x[:, :, num, start:end]
        sub_data2 = x[:, :, num_rev, start:end]
        std1 = sub_data1.std(axis=4, keepdims=True)
        std2 = sub_data2.std(axis=4, keepdims=True)
        std = (std1 * std2).mean(axis=3, keepdims=True)
        l.append(std)
    l = np.squeeze(np.array(l)).transpose(1, 2, 0)
    fct = (sub_data1.shape[4] - 1) / sub_data1.shape[4]
    
    # read cov
    cov = pd.read_parquet(output_path.replace("Corr","Cov") + name)
    cov = np.array(cov.set_index(['timestamp','ticker']))
    cov = cov.reshape(l.shape)
    # divide
    l = np.divide(cov, l, out=np.zeros_like(cov, dtype=np.float64), where=l!=0) * fct
    return l

def std(x,num,num_rev,step_list):
    """Main Function"""
    std = []
    for i in tqdm(range(len(step_list) - 1)):
        start = step_list[i]
        end = step_list[i + 1]
        sub_data1 = x[:, :, :, start:end]
        std1 = sub_data1.std(axis=3, keepdims=True)
        std.append(std1)
    return np.squeeze(np.array(std)).transpose(1, 2, 0)

def zscore(x,num,num_rev,step_list):
    """Main Function"""
    l = []
    for i in tqdm(range(len(step_list) - 1)):
        start = step_list[i]
        end = step_list[i + 1]
        sub_data1 = x[:, :, :, start:end]
        mean = sub_data1.mean(axis=3, keepdims=True)
        std = sub_data1.std(axis=3, keepdims=True)
        z_score = np.divide(mean, std, out=np.zeros_like(mean, dtype=np.float64), where=std!=0)
        l.append(z_score)
    return np.squeeze(np.array(l)).transpose(1, 2, 0)

def ret(x,num,num_rev,step_list):
    """Main Function"""
    l = []
    for i in tqdm(range(len(step_list) - 1)):
        start = step_list[i]
        end = step_list[i + 1]
        sub_data1 = x[:, :, :, start:end]
        ret = np.divide(sub_data1[:, :, :, -1], sub_data1[:, :, :, 0], out=sub_data1[:, :, :, -1], where=sub_data1[:, :, :, 0]!=0) - 1
        l.append(ret)
    return np.squeeze(np.array(l)).transpose(1, 2, 0)

def decay(x,num,num_rev,step_list):
    """Main Function"""
    l = []
    for i in tqdm(range(len(step_list) - 1)):
        start = step_list[i]
        end = step_list[i + 1]
        time_spread = end - start
        weight = np.arange(1, time_spread + 1)
        weight = weight / (weight.sum())
        sub_data1 = (x[:, :, :, start:end] * weight).sum(axis=3, keepdims=True)
        l.append(sub_data1)
    return np.squeeze(np.array(l)).transpose(1, 2, 0)

def test(input_path,name,output_path,method = None):
#     print("read" + input_path + name)
    dataframe_list = pd.read_parquet(input_path + name)
    dataframe_list = dataframe_list.set_index(["timestamp","ticker"])
    x = np.array(dataframe_list.drop('target',axis=1))
    x = x.reshape(x.shape[0], 1, day, -1)
    x = x.transpose(0, 1, 3, 2)

    feat_nums = x.shape[2]
    data_length = x.shape[3]
    num, num_rev = generate_Num_and_ReversedNum(feat_nums)
    step_list = generate_Step_List(data_length, stride)

    """Main Function"""
    if method == "Cov":
        l = cov(x,num,num_rev,step_list)
    if method == "Corr":
        l = cov(x,num,num_rev,step_list)
    if method == "Std":
        l = std(x,num,num_rev,step_list)
    if method == "Zscore":
        l = zscore(x,num,num_rev,step_list)
    if method == "Return":
        l = ret(x,num,num_rev,step_list)
    if method == "Decay":
        l = decay(x,num,num_rev,step_list)

    l = pd.DataFrame(l.reshape(l.shape[0],-1))
    l.index = dataframe_list.index
    l.columns = [str(i) for i in range(l.shape[1])]
    l.reset_index(inplace=True)
    l.to_parquet(output_path + name)
    print("save to ",output_path + name)
    return None

In [24]:
pool = mp.Pool(10)
t1 = time.time()
dir_list = ['Cov','Std','Zscore','Return','Decay']
for i in dir_list:
    for alpha in path_list:
        output = path + i + "/" 
        pool.apply_async(test, args=(data_path,alpha,output,i))   #维持执行的进程总数为10，当一个进程执行完后启动一个新进程.
pool.close()
pool.join()
t2 = time.time()
print("并行执行时间：", int(t2 - t1))

  0%|          | 0/3 [00:00<?, ?it/s]

save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Cov/2015-01-01_2015-03-01.par


 67%|██████▋   | 2/3 [00:13<00:06,  6.59s/it]

save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Cov/2016-01-01_2016-03-01.par


 67%|██████▋   | 2/3 [00:13<00:06,  6.70s/it]

...

 /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Decay/2019-09-01_2020-01-01.par


 67%|██████▋   | 2/3 [00:00<00:00,  4.85it/s]

save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Decay/2021-03-01_2021-06-01.par


100%|██████████| 3/3 [00:00<00:00,  4.84it/s]


save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Decay/2020-09-01_2021-01-01.par
并行执行时间： 105


In [25]:
t1 = time.time()
pool = mp.Pool(10)
for alpha in path_list:
    output = path + "Corr" + "/" 
    pool.apply_async(test, args=(data_path,alpha,output,"Corr"))   #维持执行的进程总数为10，当一个进程执行完后启动一个新进程.
pool.close()
pool.join()
t2 = time.time()
print("并行执行时间：", int(t2 - t1))

100%|██████████| 3/3 [00:01<00:00,  2.72it/s]


save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Corr/2015-01-01_2015-03-01.par


100%|██████████| 3/3 [00:08<00:00,  3.00s/it]


save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Corr/2017-01-01_2017-03-01.par
save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Corr/2016-01-01_2016-03-01.par


 67%|██████▋   | 2/3 [00:07<00:03,  3.79s/it]

...

save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Corr/2021-01-01_2021-03-01.par
save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Corr/2019-09-01_2020-01-01.par


 67%|██████▋   | 2/3 [00:17<00:08,  8.99s/it]

save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Corr/2020-06-01_2020-09-01.par


100%|██████████| 3/3 [00:26<00:00,  8.80s/it]


save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Corr/2021-03-01_2021-06-01.par
save to  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Corr/2020-09-01_2021-01-01.par
并行执行时间： 92


# Feat Concat ( Save to Feat_Concat file)

In [1]:
# from os import walk
# import pandas as pd
# import numpy as np
# import torch
# import torch.nn as nn
# from progressbar import ProgressBar
# from tqdm import tqdm
# import torch.utils.data as Data
# import torch.optim as optim
# from tqdm import tqdm
# from torch.autograd import Variable
# import time
# import multiprocessing as mp
# Alpha_Name = "AlphaNet_Original_Input_12.14"
# path = '/home/wuwenjun/Data/' + Alpha_Name +'/'
# target = "5d_ret"

In [26]:
os.mkdir(path+"Feat_Concat")

In [27]:
path_list = []
data_path = path + "Data/"
dataframe_list = pd.DataFrame()
for f, _, i in walk(data_path):
    for j in tqdm(i):
        path_list.append(j)
path_list.sort()
path_list

100%|██████████| 26/26 [00:00<00:00, 164730.97it/s]


['2015-01-01_2015-03-01.par',
 '2015-03-01_2015-06-01.par',
 '2015-06-01_2015-09-01.par',
 '2015-09-01_2016-01-01.par',
 '2016-01-01_2016-03-01.par',
 '2016-03-01_2016-06-01.par',
 '2016-06-01_2016-09-01.par',
 '2016-09-01_2017-01-01.par',
 '2017-01-01_2017-03-01.par',
 '2017-03-01_2017-06-01.par',
 '2017-06-01_2017-09-01.par',
 '2017-09-01_2018-01-01.par',
 '2018-01-01_2018-03-01.par',
 '2018-03-01_2018-06-01.par',
 '2018-06-01_2018-09-01.par',
 '2018-09-01_2019-01-01.par',
 '2019-01-01_2019-03-01.par',
 '2019-03-01_2019-06-01.par',
 '2019-06-01_2019-09-01.par',
 '2019-09-01_2020-01-01.par',
 '2020-01-01_2020-03-01.par',
 '2020-03-01_2020-06-01.par',
 '2020-06-01_2020-09-01.par',
 '2020-09-01_2021-01-01.par',
 '2021-01-01_2021-03-01.par',
 '2021-03-01_2021-06-01.par']

In [30]:
pool = mp.Pool(10)
t1 = time.time()
dir_list = ['Corr','Std','Zscore','Return']

final = []
def test(alpha,dir_list):
    feat_cat = []
    for i in dir_list:
        feat_cat.append(pd.read_parquet(path + i + "/" + alpha).set_index(['timestamp','ticker']))
    feat_cat = pd.concat(feat_cat,axis=1)
    feat_cat.columns = [str(i) for i in range(feat_cat.shape[1])]
    feat_cat.reset_index().to_parquet(path+"Feat_Concat/"+alpha)
    print(path+"Feat_Concat/"+alpha)
    return None

for alpha in tqdm(path_list):
#     test(alpha,dir_list)
    pool.apply_async(test, args=(alpha,dir_list))   #维持执行的进程总数为10，当一个进程执行完后启动一个新进程.
pool.close()
pool.join()

t2 = time.time()
print("并行执行时间：", int(t2 - t1))

100%|██████████| 26/26 [00:00<00:00, 55752.51it/s]


/home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Feat_Concat/2015-01-01_2015-03-01.par
/home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Feat_Concat/2016-01-01_2016-03-01.par
/home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Feat_Concat/2017-01-01_2017-03-01.par
/home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Feat_Concat/2015-03-01_2015-06-01.par
/home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Feat_Concat/2015-06-01_2015-09-01.par
/home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Feat_Concat/2017-03-01_2017-06-01.par
/home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Feat_Concat/2018-01-01_2018-03-01.par
/home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Feat_Concat/2016-03-01_2016-06-01.par
/home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Feat_Concat/2015-09-01_2016-01-01.par
/home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Feat_Concat/2017-06-01_2017-09-01.par
/home/wuwenjun/Data/

# Feat Daily Norm ( Save to Feat_Concat_Norm_Daily)

In [ ]:
# import time
# import os
# from Research.backtest.bt import BTDaily
# from Research.feature.ft import FeatureAnalysis
# import Research.utils.namespace as namespace
# import Research.utils.normalization as norm
# from Platform.database.mysql import MysqlAPI
# from Platform.utils.persistence import convert_to_standard_daily_feature_csv, convert_to_standard_daily_feature_par
# from Platform.config.mysql_info import FACTOR_LIB_MYSQL_TIO
# import DataAPI
# from os import walk
# import pandas as pd
# import numpy as np
# import torch
# import torch.nn as nn
# from progressbar import ProgressBar
# from tqdm import tqdm
# Alpha_Name = "AlphaNet_Original_Input_12.14"
# path = '/home/wuwenjun/Data/' + Alpha_Name +'/'
# target = "5d_ret"

In [ ]:
# os.mkdir(path+"Feat_Concat_Norm_Daily")

In [ ]:
# path_list = []
# data_path = path + "Data/"
# dataframe_list = pd.DataFrame()
# for f, _, i in walk(data_path):
#     for j in tqdm(i):
#         path_list.append(j)
# path_list

In [ ]:
# import multiprocessing as mp


# """ Multi Processing"""
# pool = mp.Pool(10)
# t1 = time.time()
# def test(file):
#     df = pd.read_parquet(path + "Feat_Concat/" + file)
#     final = []
#     for day in tqdm(df['timestamp'].drop_duplicates()):
#         array = df[df['timestamp'] == day].set_index(['timestamp','ticker'])
#         index = array.index
        
#         # Batch
#         array = torch.from_numpy(np.array(array)).to(torch.float) 
#         batch = nn.BatchNorm1d(array.shape[1])
#         array = batch(array)
#         array = pd.DataFrame(array.detach().numpy())
#         array.index = index
#         final.append(array)
        
#     final = pd.concat(final,axis=0)
#     final.columns = [str(i) for i in range(final.shape[1])]
#     final.reset_index(inplace=True)
#     final.to_parquet(path + "Feat_Concat_Norm_Daily/" + file)
#     print("Save to : ", path + "Feat_Concat_Norm_Daily/" + file)
#     return None

# for file in path_list:
# #     test(file)
#     pool.apply_async(test, args=(file,))   #维持执行的进程总数为10，当一个进程执行完后启动一个新进程.
# pool.close()
# pool.join()

# t2 = time.time()
# print("并行执行时间：", int(t2 - t1))

# Pooling and Norm ( Save to Pooling_Norm_Daily)

In [31]:
os.mkdir(path+"Pooling_Norm_Daily")

In [32]:
path_list = []
data_path = path + "Data/"
dataframe_list = pd.DataFrame()
for f, _, i in walk(data_path):
    for j in tqdm(i):
        path_list.append(j)
path_list.sort()
path_list

100%|██████████| 26/26 [00:00<00:00, 154683.55it/s]


['2015-01-01_2015-03-01.par',
 '2015-03-01_2015-06-01.par',
 '2015-06-01_2015-09-01.par',
 '2015-09-01_2016-01-01.par',
 '2016-01-01_2016-03-01.par',
 '2016-03-01_2016-06-01.par',
 '2016-06-01_2016-09-01.par',
 '2016-09-01_2017-01-01.par',
 '2017-01-01_2017-03-01.par',
 '2017-03-01_2017-06-01.par',
 '2017-06-01_2017-09-01.par',
 '2017-09-01_2018-01-01.par',
 '2018-01-01_2018-03-01.par',
 '2018-03-01_2018-06-01.par',
 '2018-06-01_2018-09-01.par',
 '2018-09-01_2019-01-01.par',
 '2019-01-01_2019-03-01.par',
 '2019-03-01_2019-06-01.par',
 '2019-06-01_2019-09-01.par',
 '2019-09-01_2020-01-01.par',
 '2020-01-01_2020-03-01.par',
 '2020-03-01_2020-06-01.par',
 '2020-06-01_2020-09-01.par',
 '2020-09-01_2021-01-01.par',
 '2021-01-01_2021-03-01.par',
 '2021-03-01_2021-06-01.par']

In [34]:
""" Multi Processing"""
import multiprocessing as mp
pool = mp.Pool(20)
t1 = time.time()
def test(file):
    array = pd.read_parquet(path + "Feat_Concat/" + file).set_index(["timestamp","ticker"])
    index = array.index
    array = np.array(array)
    array = torch.from_numpy(array.reshape(array.shape[0],-1,3))

    # Pooling
    max_pooling = torch.nn.MaxPool1d(kernel_size = 3, stride=1)(array) # Max Pooling
    avg_pooling = torch.nn.AvgPool1d(kernel_size = 3, stride=1)(array) # Mean Pooling
    min_pooling = -(torch.nn.MaxPool1d(kernel_size = 3, stride=1)(-array)) # MinPooling
    array = torch.cat([max_pooling,avg_pooling,min_pooling], axis=2)
    array = array.flatten(start_dim=1)
    array = pd.DataFrame(array.detach().numpy())
    array.index = index
    array.reset_index(inplace=True)
    # Daily Norm
    final = []
    for day in tqdm(array['timestamp'].drop_duplicates()):
        df = array[array['timestamp'] == day].set_index(['timestamp','ticker'])
        index = df.index
        # Batch
        df = torch.from_numpy(np.array(df)).to(torch.float) 
        batch = nn.BatchNorm1d(df.shape[1])
        df = batch(df)
        df = pd.DataFrame(df.detach().numpy())
        df.index = index
        final.append(df)

    final = pd.concat(final,axis=0)
    final.columns = [str(i) for i in range(final.shape[1])]
    final.reset_index(inplace=True)

    final.to_parquet(path + "Pooling_Norm_Daily/" + file)
    print("Save to : ", path + "Pooling_Norm_Daily/" + file)
    return None
for file in path_list:
#     test(file)
    pool.apply_async(test, args=(file,))   #维持执行的进程总数为10，当一个进程执行完后启动一个新进程.
pool.close()
pool.join()
t2 = time.time()
print("并行执行时间：", int(t2 - t1))

100%|██████████| 5/5 [00:00<00:00, 16.26it/s]


Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Pooling_Norm_Daily/2015-01-01_2015-03-01.par


 40%|███▉      | 32/81 [00:11<00:14,  3.37it/s]

Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Pooling_Norm_Daily/2017-01-01_2017-03-01.par

 62%|██████▏   | 40/65 [00:13<00:07,  3.23it/s]

...

Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Pooling_Norm_Daily/2015-06-01_2015-09-01.par


100%|██████████| 62/62 [00:04<00:00, 15.08it/s]


Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Pooling_Norm_Daily/2016-03-01_2016-06-01.par


100%|██████████| 82/82 [00:08<00:00, 10.18it/s]


Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Pooling_Norm_Daily/2016-06-01_2016-09-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Pooling_Norm_Daily/2017-03-01_2017-06-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Pooling_Norm_Daily/2017-06-01_2017-09-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Pooling_Norm_Daily/2018-03-01_2018-06-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Pooling_Norm_Daily/2019-03-01_2019-06-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Pooling_Norm_Daily/2018-06-01_2018-09-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Pooling_Norm_Daily/2019-06-01_2019-09-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Pooling_Norm_Daily/2021-01-01_2021-03-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/P

# Normalize Target and Concat

In [91]:
path_list = []
data_path = path + "Data/"
dataframe_list = pd.DataFrame()
for f, _, i in walk(data_path):
    for j in tqdm(i):
        path_list.append(j)
path_list.sort()
path_list

100%|██████████| 26/26 [00:00<00:00, 175606.93it/s]


['2015-01-01_2015-03-01.par',
 '2015-03-01_2015-06-01.par',
 '2015-06-01_2015-09-01.par',
 '2015-09-01_2016-01-01.par',
 '2016-01-01_2016-03-01.par',
 '2016-03-01_2016-06-01.par',
 '2016-06-01_2016-09-01.par',
 '2016-09-01_2017-01-01.par',
 '2017-01-01_2017-03-01.par',
 '2017-03-01_2017-06-01.par',
 '2017-06-01_2017-09-01.par',
 '2017-09-01_2018-01-01.par',
 '2018-01-01_2018-03-01.par',
 '2018-03-01_2018-06-01.par',
 '2018-06-01_2018-09-01.par',
 '2018-09-01_2019-01-01.par',
 '2019-01-01_2019-03-01.par',
 '2019-03-01_2019-06-01.par',
 '2019-06-01_2019-09-01.par',
 '2019-09-01_2020-01-01.par',
 '2020-01-01_2020-03-01.par',
 '2020-03-01_2020-06-01.par',
 '2020-06-01_2020-09-01.par',
 '2020-09-01_2021-01-01.par',
 '2021-01-01_2021-03-01.par',
 '2021-03-01_2021-06-01.par']

In [92]:
dataframe_list = pd.read_csv(path + "data.csv").set_index(["timestamp","ticker"])
df = dataframe_list[target].unstack()
df = (df.T - df.mean(axis=1)) / df.std(axis=1)
df = df.T.stack()
df = pd.DataFrame(df,columns=["target"])
# to date time
df.reset_index(inplace=True)
df['timestamp'] = pd.to_datetime(df['timestamp'])
df.set_index(["timestamp","ticker"],inplace=True)
df

target
timestamp           ticker          
2015-01-06 09:30:00 1      -1.212328
                    2      -1.374955
                    4       1.049421
                    5      -0.616170
                    6      -1.547116
...                              ...
2021-07-01 09:30:00 688700 -0.708172
                    688777  0.318826
                    688788 -0.256562
                    688819 -0.161340
                    688981 -0.073268

[5357704 rows x 1 columns]

In [93]:
pool = mp.Pool(10)
t1 = time.time()

def test(i):
    dataframe_list = pd.read_parquet(path +"Pooling_Norm_Daily/" + i).set_index(["timestamp","ticker"])
    dataframe_list = pd.merge(dataframe_list,df,how="left",left_index=True,right_index=True)
    dataframe_list.reset_index().to_parquet(path +"Pooling_Norm_Daily/" + i)
    print("Save to : ",path +"Pooling_Norm_Daily/" + i)
    return None

for file in tqdm(path_list):
    pool.apply_async(test, args=(file,))   #维持执行的进程总数为10，当一个进程执行完后启动一个新进程.
pool.close()
pool.join()

t2 = time.time()
print("并行执行时间：", int(t2 - t1))

100%|██████████| 5/5 [00:00<00:00, 47.10it/s]


Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Pooling_Norm_Daily/2015-01-01_2015-03-01.par


 57%|█████▋    | 35/61 [00:05<00:03,  6.81it/s]

Save to : 

 52%|█████▏    | 34/65 [00:05<00:04,  6.72it/s]

...

Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Pooling_Norm_Daily/2018-06-01_2018-09-01.par


 94%|█████████▎| 58/62 [00:00<00:00, 76.01it/s]

Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Pooling_Norm_Daily/2019-03-01_2019-06-01.par


100%|██████████| 62/62 [00:00<00:00, 76.82it/s]


Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Pooling_Norm_Daily/2019-06-01_2019-09-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Pooling_Norm_Daily/2021-01-01_2021-03-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Pooling_Norm_Daily/2018-09-01_2019-01-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Pooling_Norm_Daily/2020-03-01_2020-06-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Pooling_Norm_Daily/2019-09-01_2020-01-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Pooling_Norm_Daily/2020-06-01_2020-09-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Pooling_Norm_Daily/2020-09-01_2021-01-01.par
Save to :  /home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Pooling_Norm_Daily/2021-03-01_2021-06-01.par
并行执行时间： 39


In [94]:
pd.read_parquet("/home/wuwenjun/Data/AlphaNet_Original_Input_12.22_1d_return/Pooling_Norm_Daily/2015-01-01_2015-03-01.par")

,timestamp,ticker,0,1,2,3,4,5,6,7,...,179,180,181,182,183,184,185,186,187,188
0,2015-02-16 09:30:00,1,-0.024185,-0.026937,-0.025141,-0.024258,-0.026813,-0.024407,-0.023732,-0.026198,...,-0.050158,-0.299700,-0.036359,0.249617,-0.049063,-0.051807,-0.035366,-0.050169,-0.052982,-0.037844
1,2015-02-16 09:30:00,2,-0.013044,-0.005702,0.059023,-0.015676,-0.008760,0.035896,-0.014838,-0.007840,...,-0.451639,-0.322694,-0.144551,0.141399,-0.049388,-0.052816,-0.440137,-0.050494,-0.053994,-0.441627
2,2015-02-16 09:30:00,4,-0.026663,-0.029896,-0.031611,-0.026498,-0.029585,-0.032343,-0.026045,-0.028786,...,0.507348,-0.189535,0.030498,0.201037,-0.049439,-0.047686,0.526572,-0.050544,-0.048859,0.522944
3,2015-02-16 09:30:00,5,-0.026675,-0.029912,-0.031662,-0.026514,-0.029605,-0.032402,-0.026058,-0.028801,...,-1.629372,-0.312926,-0.020223,0.283267,-0.058463,-0.070131,-1.627277,-0.059570,-0.071308,-1.626200
4,2015-02-16 09:30:00,6,-0.026503,-0.029630,-0.031201,-0.026376,-0.029362,-0.032050,-0.025922,-0.028563,...,-0.963648,-0.316599,-0.203911,0.142547,-0.048793,-0.054499,-0.956212,-0.049898,-0.055673,-0.956598
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12930,2015-02-27 09:30:00,603806,-0.026665,-0.029197,-0.036605,-0.027790,-0.030408,-0.034563,-0.026774,-0.029737,...,-0.147028,-0.094642,-0.006705,0.251630,-0.143924,-0.215765,-0.207589,-0.161054,-0.233199,-0.213543
12931,2015-02-27 09:30:00,603889,-0.026664,-0.029196,-0.036603,-0.027789,-0.030407,-0.034561,-0.026773,-0.029736,...,-0.504006,-0.111578,0.010227,0.255343,-0.265476,-0.353777,-0.813651,-0.281000,-0.369479,-0.819676
12932,2015-02-27 09:30:00,603988,-0.026658,-0.029188,-0.036591,-0.027783,-0.030398,-0.034547,-0.026767,-0.029729,...,-0.542496,-0.069426,-0.229737,-0.433514,0.106457,-0.008050,-0.879005,0.086020,-0.028092,-0.885038
12933,2015-02-27 09:30:00,603993,-0.026667,-0.029199,-0.036608,-0.027793,-0.030411,-0.034564,-0.026776,-0.029740,...,-0.151529,-0.185110,-0.713055,-1.249079,-0.092173,-0.061003,-0.215233,-0.109979,-0.080370,-0.221172


In [55]:
array = pd.read_parquet("/home/wuwenjun/Data/AlphaNet_Original_Input_12.14/Feat_Concat/2015-01-01_2015-03-01.par").set_index(["timestamp","ticker"])
array.sort_values(by=["timestamp","ticker"],inplace=True)
array
# # Pooling
# max_pooling = torch.nn.MaxPool1d(kernel_size = 3, stride=1)(array) # Max Pooling
# avg_pooling = torch.nn.AvgPool1d(kernel_size = 3, stride=1)(array) # Mean Pooling
# min_pooling = -(torch.nn.MaxPool1d(kernel_size = 3, stride=1)(-array)) # MinPooling
# array = torch.cat([max_pooling,avg_pooling,min_pooling], axis=1)
# array = array.flatten(start_dim=1)
# array = pd.DataFrame(array.detach().numpy())
# array.index = index
# df = array.reset_index(inplace=True)

# # Daily Norm
# final = []
# for day in tqdm(array['timestamp'].drop_duplicates()):
#     df = array[array['timestamp'] == day].set_index(['timestamp','ticker'])
#     index = df.index
#     # Batch
#     df = torch.from_numpy(np.array(df)).to(torch.float) 
#     batch = nn.BatchNorm1d(df.shape[1])
#     df = batch(df)
#     df = pd.DataFrame(df.detach().numpy())
#     df.index = index
#     final.append(df)

# final = pd.concat(final,axis=0)
# final.columns = [str(i) for i in range(final.shape[1])]
# final.reset_index(inplace=True)

# final.to_parquet(path + "Pooling_Norm_Daily/" + file)
# print("Save to : ", path + "Pooling_Norm_Daily/" + file)

0            1            2  \
timestamp           ticker                                          
2015-02-16 09:30:00 1       1047.509016   165.116922   249.851660   
                    2       5734.296871  3871.305252  2296.206307   
                    4          1.642506     5.157167     1.294367   
                    5          0.000000     0.000000     0.000000   
                    6         72.269658    54.692880    11.679193   
...                                 ...          ...          ...   
2015-02-27 09:30:00 603806     0.990602     0.123747     0.357178   
                    603889     1.434967     0.388298     0.158693   
                    603988     4.081573     1.347522     0.415591   
                    603993     0.128444     0.110044     0.057189   
                    603998     1.295867     0.574033     0.746611   

                                      3            4            5  \
timestamp           ticker                                          
2015-02-16 09:30:00 1        924.464829   243.390500   183.194222   
                    2       4441.200376  4079.822467  1564.848673   
                    4          1.901233     6.506182     1.342306   
                    5          0.000000     0.000000     0.000000   
                    6         56.316516    53.318840     8.058444   
...                                 ...          ...          ...   
2015-02-27 09:30:00 603806     1.009622     0.105296     0.297494   
                    603889     1.513833     0.312260     0.127571   
                    603988     3.966544     1.351102     0.430010   
                    603993     0.114267     0.084144     0.068788   
                    603998     1.276227     0.565197     0.766639   

                                      6            7            8  \
timestamp           ticker                                          
2015-02-16 09:30:00 1        926.891342   158.088300    78.887819   
                    2       4471.782349  3782.803471  1117.956856   
                    4          0.990144     5.159844     0.916989   
                    5          0.000000     0.000000     0.000000   
                    6         54.038229    46.167377     6.291302   
...                                 ...          ...          ...   
2015-02-27 09:30:00 603806     1.039184     0.037416     0.267622   
                    603889     1.436000     0.261711     0.098273   
                    603988     3.653089     1.108484     0.356081   
                    603993     0.117044     0.029156     0.051223   
                    603998     1.388291     0.317899     0.748500   

                                      9  ...           314       315  \
timestamp           ticker               ...                           
2015-02-16 09:30:00 1        957.587687  ...  8.618504e+05  0.138453   
                    2       4870.817595  ...  1.659369e+06 -0.248100   
                    4          1.578639  ...  1.221067e+04 -0.025775   
                    5          0.000000  ...  0.000000e+00  0.000000   
                    6         62.654442  ...  1.295647e+05 -0.552169   
...                                 ...  ...           ...       ...   
2015-02-27 09:30:00 603806     0.979630  ...  9.607957e+03  0.951249   
                    603889     1.355527  ...  1.030385e+04  1.065205   
                    603988     3.601984  ...  4.407132e+03  2.216744   
                    603993     0.125874  ...  1.930273e+05  2.337364   
                    603998     1.282227  ...  1.310903e+04  1.358818   

                                 316       317        318       319       320  \
timestamp           ticker                                                      
2015-02-16 09:30:00 1      -0.376716  0.284385   1.522075  1.245018  0.876160   
                    2       0.016542  0.237585   2.459175  2.413624  1.709360   
                    4       0.338164  0.477255   2.458642  3.225611  1.455813   
  

In [58]:
data = pd.read_parquet("/home/wuwenjun/Data/AlphaNet_Original_Input_12.14/Cov/2015-01-01_2015-03-01.par").set_index(["timestamp","ticker"])
data.sort_values(by=["timestamp","ticker"],inplace=True)
data

0            1            2  \
timestamp           ticker                                          
2015-02-16 09:30:00 1       1047.509016   165.116922   249.851660   
                    2       5734.296871  3871.305252  2296.206307   
                    4          1.642506     5.157167     1.294367   
                    5          0.000000     0.000000     0.000000   
                    6         72.269658    54.692880    11.679193   
...                                 ...          ...          ...   
2015-02-27 09:30:00 603806     0.990602     0.123747     0.357178   
                    603889     1.434967     0.388298     0.158693   
                    603988     4.081573     1.347522     0.415591   
                    603993     0.128444     0.110044     0.057189   
                    603998     1.295867     0.574033     0.746611   

                                      3            4            5  \
timestamp           ticker                                          
2015-02-16 09:30:00 1        924.464829   243.390500   183.194222   
                    2       4441.200376  4079.822467  1564.848673   
                    4          1.901233     6.506182     1.342306   
                    5          0.000000     0.000000     0.000000   
                    6         56.316516    53.318840     8.058444   
...                                 ...          ...          ...   
2015-02-27 09:30:00 603806     1.009622     0.105296     0.297494   
                    603889     1.513833     0.312260     0.127571   
                    603988     3.966544     1.351102     0.430010   
                    603993     0.114267     0.084144     0.068788   
                    603998     1.276227     0.565197     0.766639   

                                      6            7            8  \
timestamp           ticker                                          
2015-02-16 09:30:00 1        926.891342   158.088300    78.887819   
                    2       4471.782349  3782.803471  1117.956856   
                    4          0.990144     5.159844     0.916989   
                    5          0.000000     0.000000     0.000000   
                    6         54.038229    46.167377     6.291302   
...                                 ...          ...          ...   
2015-02-27 09:30:00 603806     1.039184     0.037416     0.267622   
                    603889     1.436000     0.261711     0.098273   
                    603988     3.653089     1.108484     0.356081   
                    603993     0.117044     0.029156     0.051223   
                    603998     1.388291     0.317899     0.748500   

                                      9  ...            98         99  \
timestamp           ticker               ...                            
2015-02-16 09:30:00 1        957.587687  ...  2.883648e+05   0.116978   
                    2       4870.817595  ...  1.025901e+06   3.012376   
                    4          1.578639  ...  3.782357e+03   1.557890   
                    5          0.000000  ...  0.000000e+00   0.000000   
                    6         62.654442  ...  2.336550e+04  -0.280675   
...                                 ...  ...           ...        ...   
2015-02-27 09:30:00 603806     0.979630  ...  7.309328e+02   2.259253   
                    603889     1.355527  ...  3.310401e+03   9.444664   
                    603988     3.601984  ...  7.177707e+02  14.766100   
                    603993     0.125874  ...  1.802191e+05   2.802351   
                    603998     1.282227  ...  5.698545e+03   6.922262   

                                 100       101        102        103  \
timestamp           ticker                                             
2015-02-16 09:30:00 1      -0.507560  0.002424   0.231962  -1.006395   
                    2       0.118208 -0.861521   3.629944   0.139334   
                    4       1.926868  0.031814   2.108953   2.608465   
                    5       0.0000

In [57]:
index = array.index
array = np.array(array)
array = torch.from_numpy(array.reshape(array.shape[0],-1,3))
array[0]

tensor([[ 1.0475e+03,  1.6512e+02,  2.4985e+02],
        [ 9.2446e+02,  2.4339e+02,  1.8319e+02],
        [ 9.2689e+02,  1.5809e+02,  7.8888e+01],
        [ 9.5759e+02,  1.8629e+02,  2.2979e+02],
        [ 1.3707e+07, -8.9125e+05,  4.3135e+06],
        [-1.9883e+01, -6.5919e+00,  5.3052e-02],
        [ 1.3934e+01, -9.0615e-01,  4.3851e+00],
        [ 2.7628e+01, -1.7964e+00,  8.6947e+00],
        [ 8.8733e+02,  1.7576e+02,  1.5088e+02],
        [ 9.9600e+02,  1.9373e+02,  8.1181e+01],
        [ 1.0590e+03,  2.1103e+02,  2.0633e+02],
        [ 1.8706e+07,  3.7789e+06,  4.2384e+06],
        [-1.7389e+00, -5.8988e+00,  3.2317e+00],
        [ 1.9016e+01,  3.8414e+00,  4.3088e+00],
        [ 3.7704e+01,  7.6172e+00,  8.5435e+00],
        [ 8.3192e+02,  2.0341e+02,  1.0728e+02],
        [ 8.4682e+02,  2.1594e+02,  1.6694e+02],
        [ 1.1094e+07, -9.8365e+05,  8.6912e+05],
        [-5.6132e+00,  1.4738e+00,  2.1887e+00],
        [ 1.1278e+01, -1.0001e+00,  8.8363e-01],
        [ 2.2361e+01

In [87]:
for day in tqdm(array['timestamp'].drop_duplicates()[:1]):
    print(day)
    df = array[array['timestamp'] == day].set_index(['timestamp','ticker'])
    index = df.index
    # Batch
#     df = torch.from_numpy(np.array(df)).to(torch.float) 
#     batch = nn.BatchNorm1d(df.shape[1])
#     df = batch(df)
#     df = pd.DataFrame(df.detach().numpy())
#     df.index = index
#     final.append(df)
df

# final = pd.concat(final,axis=0)
# final.columns = [str(i) for i in range(final.shape[1])]
# final.reset_index(inplace=True)

# final.to_parquet(path + "Pooling_Norm_Daily/" + file)
# print("Save to : ", path + "Pooling_Norm_Daily/" + file)

100%|██████████| 1/1 [00:00<00:00, 43.80it/s]

2015-02-16 09:30:00


0            1            2    \
timestamp           ticker                                          
2015-02-16 09:30:00 1       1047.509016   487.492533   165.116922   
                    2       5734.296871  3967.269477  2296.206307   
                    4          5.157167     2.698013     1.294367   
                    5          0.000000     0.000000     0.000000   
                    6         72.269658    46.213910    11.679193   
...                                 ...          ...          ...   
                    603806     0.639169     0.443315     0.247522   
                    603889     5.337591     2.038748     0.359393   
                    603988     1.477298     0.991764     0.696822   
                    603993     1.009791     0.447496     0.165149   
                    603998     0.828789     0.576733     0.422953   

                                    3            4            5    \
timestamp           ticker                                          
2015-02-16 09:30:00 1        924.464829   450.349850   183.194222   
                    2       4441.200376  3361.957172  1564.848673   
                    4          6.506182     3.249907     1.342306   
                    5          0.000000     0.000000     0.000000   
                    6         56.316516    39.231267     8.058444   
...                                 ...          ...          ...   
                    603806     0.531124     0.504197     0.457222   
                    603889     4.156316     1.699826     0.272120   
                    603988     3.249069     1.584140     0.589089   
                    603993     1.040073     0.432431     0.088347   
                    603998     0.670000     0.516876     0.432567   

                                    6            7            8    \
timestamp           ticker                                          
2015-02-16 09:30:00 1        926.891342   387.955820    78.887819   
                    2       4471.782349  3124.180892  1117.956856   
                    4          5.159844     2.355659     0.916989   
                    5          0.000000     0.000000     0.000000   
                    6         54.038229    35.498969     6.291302   
...                                 ...          ...          ...   
                    603806     0.471518     0.381554     0.269633   
                    603889     3.629171     1.432856     0.253022   
                    603988     1.412584     0.811640     0.475973   
                    603993     0.961456     0.372301     0.061560   
                    603998     0.657067     0.445735     0.309842   

                                    9    ...           314       315  \
timestamp           ticker               ...                           
2015-02-16 09:30:00 1        957.587687  ...  8.618504e+05  0.284385   
                    2       4870.817595  ...  1.659369e+06  0.237585   
                    4          5.495176  ...  1.221067e+04  0.477255   
                    5          0.000000  ...  0.000000e+00  0.000000   
                    6         62.654442  ...  1.295647e+05  0.418084   
...                                 ...  ...           ...       ...   
                    603806     0.587456  ...  1.037663e+04  0.066309   
                    603889     4.535161  ...  1.162201e+04  0.222831   
                    603988     1.869058  ...  5.359328e+03  0.259096   
                    603993     1.001875  ...  2.043070e+05  1.216520   
                    603998     0.663326  ...  1.110204e+04  0.477369   

                                 316       317        318        319  \
timestamp           ticker                                             
2015-02-16 09:30:00 1       0.015374 -0.376716   1.522075   1.214418   
                    2       0.002009 -0.248100   2.459175   2.194053   
                    4       0.263215 -0.025775   3.225611   2.380022   
                    5       0.000000  0.00000

In [102]:
df[df[0] == df[0].max()]

,,0,1,2,3,4,5,6,7,8,9,...,314,315,316,317,318,319,320,321,322,323
timestamp,ticker,,,,,,,,,,,,,,,,,,,,,
2015-02-16 09:30:00,600652,2.052459e+07,7.262164e+06,298913.45616,2.002039e+07,7.177205e+06,480141.85092,1.953017e+07,6.803399e+06,32835.22632,1.973370e+07,...,81803.065636,1.998244,0.643596,-0.087245,10.549324,4.623125,1.468624,11.603702,5.085194,1.615413


In [6]:
check = pd.read_parquet("/home/wuwenjun/Data/AlphaNet_Original_Input_12.14/Data/2015-01-01_2015-03-01.par")
# check[check.index == check_index]
check.drop("target",axis=1,inplace=True)
check = np.array(check[check["ticker"] == 600652].set_index(["timestamp","ticker"]))
check = check.reshape(check.shape[0],30,9)
# check = pd.DataFrame(check[0])
check

array([[[ 3.434711e+04,  3.770068e+04,  3.402257e+04, ...,
          9.936900e+00,  7.043500e+00,  7.747500e+00],
        [ 3.699751e+04,  3.759250e+04,  3.580753e+04, ...,
         -3.299900e+00,  3.898400e+00,  4.288000e+00],
        [ 3.618616e+04,  3.667297e+04,  3.580753e+04, ...,
          0.000000e+00,  1.744700e+00,  1.919100e+00],
        ...,
        [ 4.473237e+04,  4.678779e+04,  4.419147e+04, ...,
          4.495700e+00,  1.444450e+01,  1.588820e+01],
        [ 4.597644e+04,  4.711233e+04,  4.516509e+04, ...,
         -2.325600e+00,  9.996700e+00,  1.099580e+01],
        [ 4.538145e+04,  4.619280e+04,  4.424556e+04, ...,
         -2.381000e-01,  9.403000e+00,  1.034280e+01]],

       [[ 3.699751e+04,  3.759250e+04,  3.580753e+04, ...,
         -3.299900e+00,  3.898400e+00,  4.288000e+00],
        [ 3.618616e+04,  3.667297e+04,  3.580753e+04, ...,
          0.000000e+00,  1.744700e+00,  1.919100e+00],
        [ 3.645661e+04,  3.678115e+04,  3.591571e+04, ...,
         -7.41

In [7]:
# check = check.transpose(0,2,1)
pd.DataFrame(check[0])

,0,1,2,3,4,5,6,7,8
0,34347.11,37700.68,34022.57,37700.68,36570.201301,392325.30,9.9369,7.0435,7.7475
1,36997.51,37592.50,35807.53,36456.61,36544.238135,217142.21,-3.2999,3.8984,4.2880
2,36186.16,36672.97,35807.53,36456.61,36235.384638,97182.80,0.0000,1.7447,1.9191
3,36456.61,36781.15,35915.71,36186.16,36351.677986,95150.94,-0.7418,1.7083,1.8790
4,35969.80,36510.70,35807.53,35915.71,36167.231327,73584.78,-0.7474,1.3211,1.4531
5,35969.80,35969.80,34725.73,34833.91,35253.652418,72225.59,-3.0120,1.2967,1.4263
6,34833.91,35374.81,34725.73,35104.36,35037.833600,50949.28,0.7764,0.9147,1.0061
7,35158.45,35266.63,34563.46,34996.18,34884.218200,60623.32,-0.3082,1.0884,1.1972
8,34888.00,35212.54,34779.82,35158.45,35052.437881,51584.63,0.4637,0.9261,1.0187
9,35374.81,35807.53,35104.36,35699.35,35472.175733,89323.61,1.5385,1.6036,1.7639


In [128]:
l = []
for i in tqdm(range(len(step_list) - 1)):
    start = step_list[i]
    end = step_list[i + 1]
    sub_data1 = x[:, :, num, start:end]
    sub_data2 = x[:, :, num_rev, start:end]
    mean1 = sub_data1.mean(axis=4, keepdims=True)  # (2000, 1, 36, 2, 1)
    mean2 = sub_data2.mean(axis=4, keepdims=True)
    spread1 = sub_data1 - mean1  # (2000, 1, 36, 2, 10)
    spread2 = sub_data2 - mean2
    cov = ((spread1 * spread2).sum(axis=4, keepdims=True) / (sub_data1.shape[4] - 1)).mean(axis=3,
                                                                                            keepdims=True)  # (2000, 1, 36, 1, 1)
    l.append(cov)
np.squeeze(np.array(l)).transpose(1, 2, 0)

NameError: name 'step_list' is not defined

In [18]:
check2 = pd.read_parquet("/home/wuwenjun/Data/AlphaNet_Original_Input_12.14/Decay/2015-01-01_2015-03-01.par")
check2

,timestamp,ticker,0,1,2,3,4,5,6,7,...,17,18,19,20,21,22,23,24,25,26
0,2015-02-16 09:30:00,1,1070.366909,999.556727,978.643091,1096.462182,1016.249636,992.024364,1055.225636,986.819818,...,861850.412364,0.138453,-0.376716,0.284385,1.522075,1.245018,0.876160,3.017927,2.468587,1.737222
1,2015-02-17 09:30:00,1,1054.540727,993.420364,980.813455,1083.077818,1008.988909,992.618545,1040.420000,982.427818,...,825372.445636,-1.631951,-0.601635,0.240629,1.601496,1.147820,0.839082,3.175407,2.275878,1.663689
2,2015-02-25 09:30:00,1,1038.947091,990.139091,982.660727,1065.315273,1005.177455,994.234182,1024.413818,979.301091,...,784033.056545,-1.377793,-0.151133,0.292173,1.574869,1.078356,0.797058,3.122595,2.138158,1.580369
3,2015-02-26 09:30:00,1,1026.610364,989.764182,985.308182,1057.537273,1002.102000,994.918364,1013.433091,977.131455,...,727636.666545,-0.375218,-0.482991,0.038896,1.643798,1.006125,0.739729,3.259280,1.994944,1.466684
4,2015-02-27 09:30:00,1,1022.385818,993.110727,984.804000,1050.315636,1004.788182,995.978727,1009.751091,975.800909,...,756908.456182,-0.411738,-0.287113,0.381687,1.581575,1.159289,0.769491,3.135918,2.298631,1.525691
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12930,2015-02-16 09:30:00,603998,32.997091,35.366727,33.923636,33.844364,35.976182,34.477636,32.538545,34.706727,...,11102.043636,-0.232562,-0.605060,0.477369,7.366571,9.550095,4.072662,7.366571,9.550095,4.072662
12931,2015-02-17 09:30:00,603998,32.957091,35.070000,34.074909,33.994909,35.716000,34.579455,32.603091,34.418545,...,11681.422545,0.175280,-0.519973,0.476547,8.692935,8.322640,4.285205,8.692935,8.322640,4.285205
12932,2015-02-25 09:30:00,603998,33.055091,34.862727,34.245636,34.322909,35.633455,34.742727,32.786364,34.314364,...,12217.920000,1.280062,-0.056055,0.644393,10.261596,7.895082,4.482020,10.261596,7.895082,4.482020
12933,2015-02-26 09:30:00,603998,33.480182,34.818909,34.486364,34.618909,35.608182,34.920545,33.170364,34.351455,...,12551.332727,1.144887,-0.119953,0.549635,11.622935,7.310062,4.604329,11.622935,7.310062,4.604329


In [19]:
check2[check2["ticker"] == 600652].head(30)

,timestamp,ticker,0,1,2,3,4,5,6,7,...,17,18,19,20,21,22,23,24,25,26
10705,2015-02-16 09:30:00,600652,35477.089455,36465.461455,43187.365636,35875.388364,36970.956727,44685.166000,35050.270182,35958.982545,...,587599.995636,0.019789,-0.087245,1.998244,1.468624,1.851427,10.549324,1.615413,2.036467,11.603702
10706,2015-02-17 09:30:00,600652,35344.322909,36187.143636,44023.301091,35669.846364,36706.407455,45399.153273,34672.625273,35699.352000,...,588587.095455,-0.890415,-0.511485,1.658989,1.471905,1.768620,10.567038,1.619033,1.945389,11.623189
10707,2015-02-25 09:30:00,600652,35073.874727,36014.055455,44617.306909,35461.354000,36559.872727,45878.095091,34499.538909,35558.717636,...,555166.396000,-0.393393,-0.257255,1.202387,1.416749,1.617651,9.967024,1.558356,1.779333,10.963202
10708,2015-02-26 09:30:00,600652,35011.916909,35933.412182,45111.000545,35488.890727,36799.835636,46894.001455,34515.273818,35502.660545,...,607318.682182,0.290085,0.434149,2.558155,1.508635,2.026602,10.903318,1.659418,2.229153,11.993093
10709,2015-02-27 09:30:00,600652,35108.295273,36648.383636,46096.421636,35667.879455,37512.840182,48139.053091,34664.758545,35999.304909,...,647063.561273,0.675380,2.194480,2.152520,1.694053,4.292207,11.616871,1.863369,4.721202,12.777955


In [13]:
pd.DataFrame(check[0]).iloc[:,[1]].std()

1    3980.859967
dtype: float64

In [86]:
df2 = torch.from_numpy(np.array(df)).to(torch.float) 
batch = nn.BatchNorm1d(df.shape[1])
df2 = batch(df2)
pd.DataFrame(df2.detach().numpy())
# df.index = index
# final.append(df)

,0,1,2,3,4,5,6,7,8,9,...,314,315,316,317,318,319,320,321,322,323
0,-0.024185,-0.026937,-0.025141,-0.024258,-0.026813,-0.024407,-0.023732,-0.026198,-0.020322,-0.023736,...,2.093265,-0.680457,-0.701324,-0.389046,-0.568623,-0.538389,-0.449077,-0.506007,-0.444551,-0.316227
1,-0.013044,-0.005702,0.059023,-0.015676,-0.008760,0.035896,-0.014838,-0.007840,0.042742,-0.013999,...,4.324768,-0.731170,-0.730001,-0.097460,-0.270973,-0.088982,0.114694,-0.520290,-0.355064,-0.137227
2,-0.026663,-0.029896,-0.031611,-0.026498,-0.029585,-0.032343,-0.026045,-0.028786,-0.025055,-0.026105,...,-0.284076,-0.471460,-0.169534,0.406574,-0.027530,-0.003668,-0.056865,-0.153237,-0.134309,-0.186038
3,-0.026675,-0.029912,-0.031662,-0.026514,-0.029605,-0.032402,-0.026058,-0.028801,-0.025110,-0.026119,...,-0.318242,-0.988623,-0.734312,0.465007,-1.052078,-1.095503,-1.041916,-1.295466,-1.362391,-1.284845
4,-0.026503,-0.029630,-0.031201,-0.026376,-0.029362,-0.032050,-0.025922,-0.028563,-0.024729,-0.025963,...,0.044287,-0.535579,-0.633586,-0.786814,-0.329873,-0.318804,-0.387147,-0.286877,-0.279229,-0.369902
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2582,-0.026674,-0.029910,-0.031652,-0.026512,-0.029602,-0.032382,-0.026057,-0.028799,-0.025094,-0.026118,...,-0.289208,-0.916769,-0.862839,0.071229,-0.075901,0.002367,0.128266,-0.491516,-0.450168,-0.320578
2583,-0.026662,-0.029900,-0.031648,-0.026504,-0.029595,-0.032390,-0.026049,-0.028792,-0.025095,-0.026108,...,-0.285723,-0.747159,-1.193681,-1.246767,4.862877,4.092024,1.905553,3.575917,2.947942,1.143963
2584,-0.026672,-0.029906,-0.031634,-0.026506,-0.029595,-0.032376,-0.026054,-0.028796,-0.025081,-0.026114,...,-0.303247,-0.707861,-1.219188,-0.976694,1.348438,1.117381,0.771225,0.681528,0.476302,0.209241
2585,-0.026673,-0.029909,-0.031655,-0.026511,-0.029603,-0.032398,-0.026055,-0.028799,-0.025107,-0.026117,...,0.253421,0.329624,0.464910,0.598935,-0.412176,-0.421651,-0.339631,2.996238,3.197251,3.427930


In [62]:
max_pooling = torch.nn.MaxPool1d(kernel_size = 3, stride=1)(array) # Max Pooling
avg_pooling = torch.nn.AvgPool1d(kernel_size = 3, stride=1)(array) # Mean Pooling
min_pooling = -(torch.nn.MaxPool1d(kernel_size = 3, stride=1)(-array)) # MinPooling
array = torch.cat([max_pooling,avg_pooling,min_pooling], axis=2)
array = array.flatten(start_dim=1)
array = pd.DataFrame(array.detach().numpy())
array.index = index
df = array.reset_index(inplace=True)